VacationPy

In [4]:
# dependencies
import hvplot.pandas
import pandas as pd
import requests
import json

In [5]:
# import API key
from api_keys import geoapify_api_key

In [6]:
# load the WeatherPy_Database.csv file
file_path = "/Users/grahammarsh/Documents/GitHub/python-api-challenge/Final Analysis/output_data/WeatherPy_Database.csv"
weather_df = pd.read_csv(file_path)

# display the header of the weather_df
weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Olonkinbyen,70.9221,-8.7187,-2.64,62,6,4.46,SJ,1707490169
1,1,Mogadishu,2.0371,45.3438,28.93,79,0,7.72,SO,1707490169
2,2,Port-Aux-Francais,-49.3500,70.2167,7.28,92,75,15.72,TF,1707490169
3,3,Puerto Ayora,-0.7393,-90.3518,27.53,84,72,2.24,EC,1707489943
4,4,Chonchi,-42.6128,-73.8092,19.62,47,100,2.00,CL,1707489953


Step 1: Create a map that displays a point for every city in the weather_df DataFrame. The size of the point represents the humidity level in each city.

In [7]:
# a world map depicting humidity levels for each city
weather_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    frame_height=500,
    frame_width=700,
    color="City"
)



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

Step 2: Narrow down the weather_df DataFrame to find your ideal weather condition

In [8]:
# filter the weather_df for cities with humidity levels less than 70%
low_humidity_df = weather_df.loc[weather_df["Humidity"] < 70]
low_humidity_df.head()

# drop ay rows with NaN values
low_humidity_df = low_humidity_df.dropna()
low_humidity_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Olonkinbyen,70.9221,-8.7187,-2.64,62,6,4.46,SJ,1707490169
4,4,Chonchi,-42.6128,-73.8092,19.62,47,100,2.00,CL,1707489953
10,10,Tamanrasset,22.7850,5.5228,23.95,8,0,4.12,DZ,1707489985
11,11,Ilulissat,69.2167,-51.1000,-16.99,60,40,11.83,GL,1707490170
13,13,Kingston,17.9970,-76.7936,27.53,61,20,3.09,JM,1707490010


Step 3: Create a new DataFrame called hotel_df

In [30]:
# use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates and humidity
hotel_df = low_humidity_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# add a new column to the hotel_df called "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()


(194, 6)

Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

In [ ]:
# define the search categories
categories = "accommodation.hotel"

# define the search radius
radius = 10000

# define the search limit
limit = 1

# define the parameters dictionary
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_api_key
}

# print a message to follow up the hotel search
print("Starting hotel search")

# iterate through the hotel_df and retrieve the hotel name for each city
for index, row in hotel_df.iterrows():
    # get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add the latitude and longitude to the parameters dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # set the base URL for the Geoapify API
    base_url = "https://api.geoapify.com/v2/places"

    # run the API request using the parameters dictionary and convert the response to JSON
    name_address = requests.get(base_url, params=params).json()
    
    # get the first hotel name from the response and store it in the hotel_df
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (IndexError, KeyError):
        hotel_df.loc[index, "Hotel Name"] = "Hotel not found"

    # log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# display the hotel_df
hotel_df


Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
# configure the map plot
